<a href="https://colab.research.google.com/github/hamzafarooq/multi-agent-course/blob/main/Module_3_Agentic_RAG/Agentic_RAG_with_Semantic_Cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic RAG with Semantic Cache

This notebook combines two powerful concepts:

- **Semantic Cache** — A FAISS-backed cache that stores previous query embeddings and their answers. When a new query is semantically similar to a cached one, the stored answer is returned instantly — no LLM or API call needed.
- **Agentic RAG** — An intelligent retrieval system that routes queries to the right knowledge source: OpenAI documentation (via Qdrant), 10-K financial filings (via Qdrant), or live internet search (via SerpApi).

## Architecture

```
User Query
    │
    ▼
┌─────────────────────────────┐
│  Is query time-sensitive?   │  ──YES──▶  Agentic RAG (no caching)
│  (current events, "today",  │
│   live data, etc.)          │
└──────────────┬──────────────┘
               │ NO
               ▼
┌─────────────────────────────┐
│   Semantic Cache Lookup     │  ──HIT──▶  Return cached answer ⚡
│   (FAISS similarity search) │
└──────────────┬──────────────┘
               │ MISS
               ▼
┌─────────────────────────────┐
│      Agentic RAG Router     │
│   (GPT-4o classifies query) │
└──────┬──────────┬───────────┘
       │          │           │
  OPENAI      10K_DOC    INTERNET
  QUERY       QUERY       QUERY
    │            │            │
  Qdrant      Qdrant      SerpApi
  (RAG)       (RAG)      (live web)
       │          │           │
       └──────────┴───────────┘
               │
               ▼
    Store answer in cache 💾
               │
               ▼
          Return answer
```

## Why combine them?

- **Speed**: Cached answers return in milliseconds vs. 2–5 seconds for full RAG.
- **Cost**: Fewer LLM and API calls for repeated or similar questions.
- **Correctness**: Time-sensitive queries (e.g., *"What happened today?"*) always bypass the cache to ensure fresh answers.

## 1. Setup and Dependencies

In [1]:
!pip install -U faiss-cpu sentence_transformers transformers openai qdrant_client python-dotenv

In [2]:
import faiss                          # Efficient vector similarity search
import json                           # JSON persistence for cache
import numpy as np                    # Numerical operations on embeddings
import requests                       # HTTP calls to SerpApi and Qdrant
import re                             # Regex for parsing LLM JSON responses
import time                           # Latency measurement
import asyncio                        # Async support for Qdrant retrieval
import nest_asyncio                   # Allow nested event loops in Colab/Jupyter
import os

from sentence_transformers import SentenceTransformer      # Nomic embed model wrapper
from transformers import AutoTokenizer, AutoModel          # Tokenizer + model for RAG embeddings
from openai import OpenAI, OpenAIError                     # OpenAI client for routing + generation
import qdrant_client                                       # Qdrant vector database client
from qdrant_client import models

nest_asyncio.apply()  # Allow asyncio.run() inside Jupyter/Colab

## 2. API Keys

**On Google Colab** — store keys in the Secrets panel (`🔑` icon, left sidebar):
| Secret name | Where to get it |
|---|---|
| `SERP_API_KEY` | [serpapi.com](https://serpapi.com) |
| `OPENAI_API_KEY` | [platform.openai.com](https://platform.openai.com) |

**Running locally** — add keys to `Module_3_Agentic_RAG/.env`:
```
serp_api_key=<your_key>
openai_api_key=<your_key>
```
The cell below detects the environment automatically and loads from the right source.

In [3]:
# ── Credential loading — works on Colab and locally ──────────────────────────
try:
    from google.colab import userdata
    serp_api_key   = userdata.get('SERP_API_KEY')
    openai_api_key = userdata.get('OPENAI_API_KEY')
    print("Running on Colab — credentials loaded from Secrets.")
except ImportError:
    from dotenv import load_dotenv, find_dotenv
    load_dotenv(find_dotenv())  # walks up the directory tree to find .env
    serp_api_key   = os.getenv("serp_api_key")   or os.getenv("SERP_API_KEY")
    openai_api_key = os.getenv("openai_api_key") or os.getenv("OPENAI_API_KEY")
    print("Running locally — credentials loaded from .env file.")

print(f"SerpApi key loaded:    {'✅' if serp_api_key else '❌ MISSING'}")
print(f"OpenAI API key loaded: {'✅' if openai_api_key else '❌ MISSING'}")

# Initialize OpenAI client — used for query routing and RAG response generation
openaiclient = OpenAI(api_key=openai_api_key)

Running on Colab — credentials loaded from Secrets.
SerpApi key loaded:    ✅
OpenAI API key loaded: ✅


## 3. Semantic Cache with Time-Sensitivity Filter

The `SemanticCaching` class provides:

- **`is_time_sensitive(question)`** — Detects questions that should never be cached because their answers change over time (e.g., *"What is the weather today?"*, *"Latest AI news this week"*). These go directly to the Agentic RAG pipeline for a fresh answer.
- **`check_cache(question)`** — Embeds the query and checks FAISS for a near-neighbor within the similarity threshold. Returns hit/miss, the cached answer if available, and the computed embedding (reused on cache miss to avoid re-encoding).
- **`add_to_cache(question, answer, embedding)`** — Stores a new entry after a successful RAG response.

### What counts as time-sensitive?

A question is flagged as time-sensitive if it contains temporal indicators such as:
> `today`, `now`, `current`, `latest`, `this week`, `yesterday`, `upcoming`, `live`, `breaking`, `stock price`, `weather`, `forecast`, `real-time`, `last month`, `this quarter` …

These questions are intentionally excluded from caching because a cached answer from last week would be incorrect today.

In [4]:
class SemanticCaching:
    """
    A semantic cache backed by FAISS that:
    - Detects and bypasses time-sensitive queries
    - Returns cached answers for semantically similar past queries
    - Persists cache entries to disk as JSON
    """

    # Keywords that indicate a question is time-sensitive and should NOT be cached
    TIME_SENSITIVE_KEYWORDS = [
        "today", "tonight", "now", "currently", "current",
        "latest", "recent", "recently", "right now", "at the moment",
        "at present", "as of now", "this week", "this month", "this year",
        "this quarter", "this season", "this morning", "this afternoon",
        "this evening", "this weekend", "yesterday", "tomorrow",
        "last week", "last month", "last year", "upcoming", "live",
        "breaking", "just happened", "what time", "what day", "what date",
        "happening now", "events today", "news today", "news this week",
        "stock price", "share price", "weather", "forecast", "temperature",
        "real-time", "realtime", "right now", "schedule today",
    ]

    def __init__(self, json_file='rag_cache.json', threshold=0.2, clear_on_init=False):
        """
        Args:
            json_file (str): Path to the JSON file for cache persistence.
            threshold (float): Max Euclidean distance to count as a cache hit (lower = stricter).
            clear_on_init (bool): If True, wipe existing cache on startup.
        """
        self.embedding_dim = 768
        self.index = faiss.IndexFlatL2(self.embedding_dim)
        self.euclidean_threshold = threshold
        self.json_file = json_file

        # Load the Nomic embedding model for query encoding
        print("Loading Nomic embedding model...")
        self.encoder = SentenceTransformer(
            'nomic-ai/nomic-embed-text-v1.5',
            trust_remote_code=True
        )
        print("Embedding model ready.")

        if clear_on_init:
            self.clear_cache()
        else:
            self.load_cache()

    # ------------------------------------------------------------------
    # Time-sensitivity detection
    # ------------------------------------------------------------------

    def is_time_sensitive(self, question: str) -> bool:
        """
        Returns True if the question references time-dependent information
        that should NOT be cached (answers change frequently).

        Examples that return True:
            'What is the weather today?'
            'Who won the game last night?'
            'What are the latest AI news this week?'
            'What is the current stock price of Apple?'

        Examples that return False (safe to cache):
            'What is the capital of France?'
            'Who founded Apple?'
            'What is machine learning?'
            'What was Uber revenue in 2021?'
        """
        question_lower = question.lower()
        return any(keyword in question_lower for keyword in self.TIME_SENSITIVE_KEYWORDS)

    # ------------------------------------------------------------------
    # Cache persistence
    # ------------------------------------------------------------------

    def clear_cache(self):
        """Reset all in-memory state and overwrite the JSON file."""
        self.cache = {'questions': [], 'embeddings': [], 'response_text': []}
        self.index = faiss.IndexFlatL2(self.embedding_dim)
        self.save_cache()
        print("Semantic cache cleared.")

    def load_cache(self):
        """Load cache from JSON and rebuild the FAISS index from stored embeddings."""
        try:
            with open(self.json_file, 'r') as f:
                self.cache = json.load(f)
            # Rebuild FAISS index so lookups work after a cold start
            if self.cache['embeddings']:
                embeddings = np.array(self.cache['embeddings'], dtype=np.float32)
                self.index.add(embeddings)
            print(f"Cache loaded: {len(self.cache['questions'])} entries.")
        except FileNotFoundError:
            self.cache = {'questions': [], 'embeddings': [], 'response_text': []}
            print("No existing cache found — starting fresh.")

    def save_cache(self):
        """Persist the cache to disk."""
        with open(self.json_file, 'w') as f:
            json.dump(self.cache, f)

    # ------------------------------------------------------------------
    # Cache lookup and storage
    # ------------------------------------------------------------------

    def check_cache(self, question: str):
        """
        Encode the question and search the FAISS index for a near-enough neighbor.

        Returns:
            tuple: (
                hit (bool),           -- True if a cached answer was found
                answer (str|None),    -- Cached answer text, or None on miss
                embedding (np.array), -- The computed embedding (reuse on cache miss)
                similarity (float|None), -- Similarity score [0,1], or None on miss
                row_id (int|None),    -- Index of the matched row, or None on miss
            )
        """
        embedding = self.encoder.encode([question], normalize_embeddings=True)

        # Nothing in the index yet — guaranteed miss
        if self.index.ntotal == 0:
            return False, None, embedding, None, None

        D, I = self.index.search(embedding, 1)

        if I[0][0] != -1 and D[0][0] <= self.euclidean_threshold:
            row_id = int(I[0][0])
            similarity = float(1.0 - D[0][0])  # Convert distance → similarity score
            return True, self.cache['response_text'][row_id], embedding, similarity, row_id

        return False, None, embedding, None, None

    def add_to_cache(self, question: str, answer: str, embedding: np.ndarray):
        """
        Store a new question-answer pair.

        Args:
            question (str): The original user query.
            answer (str): The answer from the Agentic RAG pipeline.
            embedding (np.ndarray): Pre-computed query embedding (shape: [1, 768]).
        """
        self.cache['questions'].append(question)
        self.cache['embeddings'].append(embedding[0].tolist())
        self.cache['response_text'].append(answer)
        self.index.add(embedding)
        self.save_cache()

In [16]:
# Instantiate the semantic cache
# Set clear_on_init=True to wipe any previously stored entries
cache = SemanticCaching(json_file='rag_cache.json', threshold=0.2, clear_on_init=True)

Loading Nomic embedding model...


Embedding model ready.
Semantic cache cleared.


## 4. Agentic RAG Components

These are the building blocks of the Agentic RAG pipeline, adapted from the `Agentic_RAG_Notebook.ipynb`.

### Components
| Component | Purpose |
|---|---|
| `get_internet_content()` | Fetches real-time answers via SerpApi (Google search) |
| `route_query()` | Uses GPT-4o to classify the query into `OPENAI_QUERY`, `10K_DOCUMENT_QUERY`, or `INTERNET_QUERY` |
| `get_text_embeddings()` | Converts text to dense vectors using Nomic embed model |
| `rag_formatted_response()` | Generates a grounded answer from retrieved Qdrant chunks |
| `retrieve_and_response()` | Async: embeds query → searches Qdrant → calls RAG generator |

In [17]:
def get_internet_content(user_query: str, action: str = "INTERNET_QUERY") -> str:
    """
    Search Google in real time using SerpApi and return a formatted answer.

    Used for INTERNET_QUERY routes — queries that need live web information
    beyond what is available in the Qdrant document collections.

    Args:
        user_query (str): The user's question.
        action (str): Route label (always "INTERNET_QUERY" here).

    Returns:
        str: Formatted string combining Google's answer box and top organic
             result snippets, or an error message.
    """
    print("Getting your response from the internet 🌐 ...")

    if not serp_api_key:
        return "Error: Missing SERP_API_KEY — add it to Colab Secrets or .env"

    params = {
        "q": user_query,
        "api_key": serp_api_key,
        "engine": "google",
        "num": 5,
    }

    try:
        response = requests.get("https://serpapi.com/search.json", params=params)
        response.raise_for_status()
        data = response.json()

        parts = []

        # Answer box — Google's highlighted direct answer (most relevant)
        answer_box = data.get("answer_box", {})
        if answer_box.get("answer"):
            parts.append(f"[Direct Answer] {answer_box['answer']}")
        elif answer_box.get("snippet"):
            parts.append(f"[Direct Answer] {answer_box['snippet']}")

        # Top organic results — titles + snippets
        for i, result in enumerate(data.get("organic_results", [])[:5], start=1):
            title   = result.get("title", "")
            snippet = result.get("snippet", "")
            link    = result.get("link", "")
            if snippet:
                parts.append(f"[{i}] {title}\n    {snippet}\n    Source: {link}")

        return "\n\n".join(parts) if parts else "No results found."

    except requests.exceptions.RequestException as e:
        return f"SerpApi request error: {e}"
    except Exception as e:
        return f"Unexpected error: {e}"

In [18]:
def route_query(user_query: str) -> dict:
    """
    Classify the user query into one of three categories using GPT-4o.

    Returns:
        dict with keys:
            'action': 'OPENAI_QUERY' | '10K_DOCUMENT_QUERY' | 'INTERNET_QUERY'
            'reason': brief justification
            'answer': short answer if applicable, else empty string
    """
    router_system_prompt = f"""
    As a professional query router, classify user input into one of three categories:
    1. "OPENAI_QUERY": Questions about OpenAI's documentation — agents, tools, APIs, models, embeddings, guardrails.
    2. "10K_DOCUMENT_QUERY": Questions about company financials, 10-K annual reports, Uber or Lyft filings.
    3. "INTERNET_QUERY": Everything else — general knowledge, comparisons, trends, or anything not in internal docs.

    Always respond in this valid JSON format:
    {{
        "action": "OPENAI_QUERY" or "10K_DOCUMENT_QUERY" or "INTERNET_QUERY",
        "reason": "brief justification",
        "answer": "AT MAX 5 words. Leave empty if INTERNET_QUERY"
    }}

    User: {user_query}
    """
    try:
        response = openaiclient.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": router_system_prompt}]
        )
        task_response = response.choices[0].message.content
        json_match = re.search(r"\{.*\}", task_response, re.DOTALL)
        return json.loads(json_match.group())
    except (OpenAIError, json.JSONDecodeError, AttributeError) as err:
        return {"action": "INTERNET_QUERY", "reason": f"Routing error: {err}", "answer": ""}

### Qdrant Vector Database Setup

We load the pre-built Qdrant collections from the course repository. Two collections are available:
- `opnai_data` — OpenAI Agents documentation
- `10k_data` — Uber 2021 and Lyft 2024 10-K filings

In [19]:
import os

# Resolve the path to the prebuilt Qdrant vector database.
# On Colab: clone the repo if needed, then point to /content/multi-agent-course/...
# Locally:  the qdrant_data folder is already present in the cloned repo next to this notebook.

try:
    from google.colab import userdata  # will only succeed on Colab
    QDRANT_PATH = "/content/multi-agent-course/Module_3_Agentic_RAG/Agentic_RAG/qdrant_data"
    if not os.path.exists("/content/multi-agent-course"):
        import subprocess
        subprocess.run(["git", "clone", "https://github.com/hamzafarooq/multi-agent-course.git",
                        "/content/multi-agent-course"], check=True)
        print("Repository cloned.")
    else:
        print("Repository already present.")
except ImportError:
    # Running locally — qdrant_data is in the Agentic_RAG sub-folder
    NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
    QDRANT_PATH = os.path.join(NOTEBOOK_DIR, "Agentic_RAG", "qdrant_data")
    print(f"Running locally — Qdrant path: {QDRANT_PATH}")

print(f"Qdrant data path: {QDRANT_PATH}")

Repository already present.
Qdrant data path: /content/multi-agent-course/Module_3_Agentic_RAG/Agentic_RAG/qdrant_data


In [9]:
# Initialize async Qdrant client using the resolved path from the cell above
qdrant = qdrant_client.AsyncQdrantClient(path=QDRANT_PATH)

In [10]:
# Load tokenizer and model for query embedding during Qdrant retrieval
# Note: The Nomic model is already loaded inside SemanticCaching above.
# We use AutoTokenizer/AutoModel here to match the Agentic RAG notebook's approach
# (mean-pooling over token embeddings, without normalization — different from SentenceTransformer).
text_tokenizer = AutoTokenizer.from_pretrained(
    "nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True
)
text_model = AutoModel.from_pretrained(
    "nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True
)

def get_text_embeddings(text: str) -> np.ndarray:
    """
    Embed a text string using mean pooling over token embeddings.
    Used for Qdrant retrieval queries.

    Args:
        text (str): Input text to embed.

    Returns:
        np.ndarray: 1-D embedding vector of shape (768,).
    """
    inputs = text_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = text_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings[0].detach().numpy()

In [11]:
def rag_formatted_response(user_query: str, context: list) -> str:
    """
    Generate a grounded answer from retrieved document chunks.
    Citations are formatted as [1][2]... matching chunk order.

    Args:
        user_query (str): The user's original question.
        context (list): List of text chunks retrieved from Qdrant.

    Returns:
        str: Model-generated answer with numbered citations.
    """
    rag_prompt = f"""
    Based on the given context, answer the user query: {user_query}\nContext:\n{context}
    Employ references to the ID of articles provided [ID], ensuring their relevance to the query.
    The referencing should always be in the format of [1][2]... etc.
    """
    response = openaiclient.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": rag_prompt}]
    )
    return response.choices[0].message.content

In [12]:
async def retrieve_and_response(user_query: str, action: str) -> str:
    """
    Retrieve relevant chunks from the appropriate Qdrant collection
    and generate a RAG response.

    Args:
        user_query (str): The user's question.
        action (str): 'OPENAI_QUERY' or '10K_DOCUMENT_QUERY'.

    Returns:
        str: Generated answer grounded in retrieved document chunks.
    """
    collections = {
        "OPENAI_QUERY": "opnai_data",
        "10K_DOCUMENT_QUERY": "10k_data",
    }
    if action not in collections:
        return "Invalid action type for retrieval."

    try:
        query_embedding = get_text_embeddings(user_query)
    except Exception as e:
        return f"Embedding error: {e}"

    try:
        text_hits = await qdrant.query_points(
            collection_name=collections[action],
            query=query_embedding,
            limit=3
        )
    except Exception as e:
        return f"Qdrant query error: {e}"

    contents = [point.payload['content'] for point in text_hits.points]
    if not contents:
        return "No relevant content found in the database."

    try:
        return rag_formatted_response(user_query, contents)
    except Exception as e:
        return f"RAG generation error: {e}"

## 5. Combining Agentic RAG with Semantic Cache

The `agentic_rag_with_cache()` function is the main entry point. It:

1. **Checks time-sensitivity** — time-dependent questions always get a fresh answer.
2. **Checks the semantic cache** — similar past queries return instantly.
3. **Runs full Agentic RAG** on cache misses via `_get_rag_result()`, which:
   - Routes the query (GPT-4o → `OPENAI_QUERY` / `10K_DOCUMENT_QUERY` / `INTERNET_QUERY`)
   - Calls the appropriate retrieval or search function
   - Returns the answer as a string
4. **Stores the result** in the cache for future use.

In [13]:
# Maps routing labels to their handler functions
routes = {
    "OPENAI_QUERY": retrieve_and_response,
    "10K_DOCUMENT_QUERY": retrieve_and_response,
    "INTERNET_QUERY": get_internet_content,
}


def _get_rag_result(user_query: str) -> str:
    """
    Run the full Agentic RAG pipeline and return the answer string.

    This is a helper used by agentic_rag_with_cache() so the result can be
    captured, displayed, and stored in the semantic cache.

    Args:
        user_query (str): The user's question.

    Returns:
        str: The final answer from the chosen knowledge source.
    """
    GREY = "\033[90m"
    RESET = "\033[0m"

    # Step 1: Route the query
    route_response = route_query(user_query)
    action = route_response.get("action", "INTERNET_QUERY")
    reason = route_response.get("reason", "")

    print(f"{GREY}📍 Route: {action}")
    print(f"📝 Reason: {reason}{RESET}")

    # Step 2: Call the appropriate handler
    route_fn = routes.get(action)
    if not route_fn:
        return f"Unsupported action: {action}"

    try:
        if action in ("OPENAI_QUERY", "10K_DOCUMENT_QUERY"):
            result = asyncio.run(route_fn(user_query, action))
        else:
            result = route_fn(user_query, action)
    except Exception as e:
        result = f"Execution error: {e}"

    return result

In [14]:
def agentic_rag_with_cache(user_query: str) -> str:
    """
    Main entry point: Agentic RAG with semantic cache layer.

    Query flow:
        1. If time-sensitive → skip cache, run Agentic RAG directly
        2. Check semantic cache → if hit, return cached answer instantly
        3. Cache miss → run Agentic RAG, store answer, return answer

    Args:
        user_query (str): The user's question.

    Returns:
        str: The answer text.
    """
    CYAN  = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    BOLD  = "\033[1m"
    RESET = "\033[0m"

    print(f"{BOLD}{CYAN}👤 User Query:{RESET} {user_query}\n")

    # ------------------------------------------------------------------ #
    # Step 1: Time-sensitivity check                                       #
    # ------------------------------------------------------------------ #
    if cache.is_time_sensitive(user_query):
        print(
            f"{YELLOW}⏰ Time-sensitive query detected — bypassing semantic cache "
            f"to ensure a fresh answer.{RESET}\n"
        )
        result = _get_rag_result(user_query)
        print(f"\n{BOLD}{CYAN}🤖 Response (live):{RESET}\n{result}\n")
        return result

    # ------------------------------------------------------------------ #
    # Step 2: Semantic cache lookup                                        #
    # ------------------------------------------------------------------ #
    start_time = time.time()
    hit, cached_answer, embedding, similarity, row_id = cache.check_cache(user_query)

    if hit:
        elapsed = time.time() - start_time
        print(
            f"{GREEN}✅ Semantic Cache HIT{RESET} "
            f"(row {row_id}, similarity: {similarity:.3f}, time: {elapsed:.3f}s)\n"
        )
        print(f"{BOLD}{CYAN}🤖 Response (cached):{RESET}\n{cached_answer}\n")
        return cached_answer

    # ------------------------------------------------------------------ #
    # Step 3: Cache miss → run Agentic RAG                                 #
    # ------------------------------------------------------------------ #
    print(f"{YELLOW}❌ Semantic Cache MISS — running Agentic RAG pipeline...{RESET}\n")
    result = _get_rag_result(user_query)

    # ------------------------------------------------------------------ #
    # Step 4: Store the answer for future cache hits                       #
    # ------------------------------------------------------------------ #
    cache.add_to_cache(user_query, result, embedding)
    print(f"\n{GREEN}💾 Answer cached for future similar queries.{RESET}")

    print(f"\n{BOLD}{CYAN}🤖 Response:{RESET}\n{result}\n")
    return result

## 6. Testing the Combined System

We run a series of queries to demonstrate all three pathways:

| Query | Expected path |
|---|---|
| *"What was Uber's revenue in 2021?"* | Cache MISS → 10K RAG → stored |
| *"How much did Uber earn in 2021?"* | Cache HIT (semantically similar to above) |
| *"How to use the OpenAI Agents SDK?"* | Cache MISS → OpenAI RAG → stored |
| *"What are the best AI tools this week?"* | Time-sensitive → bypass cache → SerpApi |
| *"What is the current stock price of Apple?"* | Time-sensitive → bypass cache → SerpApi |
| *"List new LLMs in 2025"* | Cache MISS → INTERNET → SerpApi → stored |

In [20]:
# Test 1: Cache MISS — should route to 10K_DOCUMENT_QUERY and store result
result = agentic_rag_with_cache("What was Uber's revenue in 2021?")

👤 User Query: What was Uber's revenue in 2021?

❌ Semantic Cache MISS — running Agentic RAG pipeline...

📍 Route: 10K_DOCUMENT_QUERY
📝 Reason: The question relates to Uber's financials.

💾 Answer cached for future similar queries.

🤖 Response:
Uber's revenue in 2021 was $3,208,323,000 [1].



In [21]:
# Test 2: Cache HIT — semantically similar to Test 1, returns instantly from cache
result = agentic_rag_with_cache("How much did Uber earn in fiscal year 2021?")

👤 User Query: How much did Uber earn in fiscal year 2021?

✅ Semantic Cache HIT (row 0, similarity: 0.838, time: 0.123s)

🤖 Response (cached):
Uber's revenue in 2021 was $3,208,323,000 [1].



In [22]:
# Test 3: Cache MISS — routes to OPENAI_QUERY and stores result
result = agentic_rag_with_cache("How do I build an agent with the OpenAI Agents SDK?")

👤 User Query: How do I build an agent with the OpenAI Agents SDK?

❌ Semantic Cache MISS — running Agentic RAG pipeline...

📍 Route: OPENAI_QUERY
📝 Reason: Question about OpenAI's documentation on agents.

💾 Answer cached for future similar queries.

🤖 Response:
To build an agent with the OpenAI Agents SDK, you need to focus on three core components: Model, Tools, and Instructions. Here's a step-by-step guide using the SDK:

1. **Model Selection**: Choose an appropriate LLM for your agent by evaluating task complexity, latency, and cost. Start with the most capable model to establish a performance baseline, then experiment with smaller models to optimize for cost and speed [1][2].

2. **Define Tools**: Tools are external functions or APIs the agent can utilize to perform actions. Define these tools clearly for the agent to use, such as a `get_weather` function in the example provided [1].

3. **Provide Instructions**: Write explicit guidelines and guardrails that define agent behavior.

In [23]:
# Test 4: Time-sensitive query — BYPASSES cache, calls Ares API for live answer
result = agentic_rag_with_cache("What are the best AI tools this week?")

👤 User Query: What are the best AI tools this week?

⏰ Time-sensitive query detected — bypassing semantic cache to ensure a fresh answer.

📍 Route: INTERNET_QUERY
📝 Reason: General inquiry about AI tools.
Getting your response from the internet 🌐 ...

🤖 Response (live):
[1] The Best AI Tools for 2026
    Without a doubt, ChatGPT, Gemini, and Claude are the best AI tools to date. They can provide answers to your everyday questions, do web searches ...
    Source: https://medium.com/artificial-corner/the-best-ai-tools-for-2026-933535a44f8b

[2] I tried 70+ best AI tools in 2026
    I went deep into each tool, from image generation to email automation, chatbot building to scheduling assistants.
    Source: https://www.techradar.com/best/best-ai-tools

[3] I Tested 47 AI Tools in 30 Days — Here Are the Only 7 Worth ...
    I Tested 47 AI Tools in 30 Days — Here Are the Only 7 Worth Paying For How I cut my workday from 10 hours to 6 without sacrificing quality ...
    Source: https://levelu

In [24]:
# Test 5: Time-sensitive query — stock price, always fetched live
result = agentic_rag_with_cache("What is the current stock price of Apple?")

👤 User Query: What is the current stock price of Apple?

⏰ Time-sensitive query detected — bypassing semantic cache to ensure a fresh answer.

📍 Route: INTERNET_QUERY
📝 Reason: Question on real-time stock price
Getting your response from the internet 🌐 ...

🤖 Response (live):
[1] Stock Price - Apple - Investor Relations
    Stock Quote: NASDAQ: AAPL ; Day's Open262.60 ; Closing Price260.58 ; Volume30.8 ; Intraday High264.48 ; Intraday Low260.05.
    Source: https://investor.apple.com/stock-price/default.aspx

[2] AAPL: Apple Inc - Stock Price, Quote and News
    Apple Inc AAPL:NASDAQ ; Open258.97 ; Day High264.75 ; Day Low258.16 ; Prev Close260.58 ; 52 Week High288.62 ...
    Source: https://www.cnbc.com/quotes/AAPL

[3] Buy or Sell Apple Stock - AAPL Stock Price Quote & News
    Shares are currently priced at $261.38, which is +1.2% above the low and -1.3% below the high. Apple(AAPL) shares are trading with a volume of 42.05M, against a ...
    Source: https://robinhood.com/us/en/stoc

In [25]:
# Test 6: Cache MISS — INTERNET_QUERY, stored after Ares API call
result = agentic_rag_with_cache("What are the most popular open-source LLMs?")

👤 User Query: What are the most popular open-source LLMs?

❌ Semantic Cache MISS — running Agentic RAG pipeline...

📍 Route: INTERNET_QUERY
📝 Reason: General inquiry about open-source LLMs.
Getting your response from the internet 🌐 ...

💾 Answer cached for future similar queries.

🤖 Response:
[1] Open LLM Leaderboard 2025
    This LLM leaderboard displays the latest public benchmark performance for SOTA open-sourced model versions released after April 2024.
    Source: https://www.vellum.ai/open-llm-leaderboard

[2] Top 10 open source LLMs for 2025
    Top open source LLMs in 2024 · 1. LLaMA 3 · 2. Google Gemma 2 · 3. Command R+ · 4. Mistral-8x22b · 5. Falcon 2 · 6. Grok 1.5 · 7. Qwen1.5 · 8. BLOOM.
    Source: https://www.instaclustr.com/education/open-source-ai/top-10-open-source-llms-for-2025/

[3] I locally benchmarked 41 open-source LLMs across 19 ...
    Hello everyone! I benchmarked 41 open-source LLMs using lm-evaluation-harness. Here are the 19 tasks covered:.
    Source: http

In [26]:
# Test 7: Cache HIT — similar to Test 6
result = agentic_rag_with_cache("Which open-source large language models are most widely used?")

👤 User Query: Which open-source large language models are most widely used?

❌ Semantic Cache MISS — running Agentic RAG pipeline...

📍 Route: INTERNET_QUERY
📝 Reason: Question on general open-source LLMs.
Getting your response from the internet 🌐 ...

💾 Answer cached for future similar queries.

🤖 Response:
[1] Top 10 open source LLMs for 2025
    Unlike proprietary models developed by companies like OpenAI and Google, open source LLMs are licensed to be freely used, modified, and distributed by anyone.
    Source: https://www.instaclustr.com/education/open-source-ai/top-10-open-source-llms-for-2025/

[2] 9 Top Open-Source LLMs for 2026 and Their Uses
    LLM are the foundation models of popular and widely-used chatbots, like ChatGPT and Google Gemini. ... most powerful open-source large language ...
    Source: https://www.datacamp.com/blog/top-open-source-llms

[3] The best open source large language model
    The largest open-source models, DeepSeek-V3 and DeepSeek-R1, match GPT-4o

## 7. Inspecting the Cache

View the current state of the semantic cache — what questions have been cached and what answers are stored.

In [27]:
print(f"Total cached entries: {len(cache.cache['questions'])}")
print(f"FAISS index size: {cache.index.ntotal}\n")

for i, (q, a) in enumerate(zip(cache.cache['questions'], cache.cache['response_text'])):
    print(f"[{i}] Q: {q}")
    print(f"    A: {a[:120]}...\n" if len(a) > 120 else f"    A: {a}\n")

Total cached entries: 4
FAISS index size: 4

[0] Q: What was Uber's revenue in 2021?
    A: Uber's revenue in 2021 was $3,208,323,000 [1].

[1] Q: How do I build an agent with the OpenAI Agents SDK?
    A: To build an agent with the OpenAI Agents SDK, you need to focus on three core components: Model, Tools, and Instructions...

[2] Q: What are the most popular open-source LLMs?
    A: [1] Open LLM Leaderboard 2025
    This LLM leaderboard displays the latest public benchmark performance for SOTA open-so...

[3] Q: Which open-source large language models are most widely used?
    A: [1] Top 10 open source LLMs for 2025
    Unlike proprietary models developed by companies like OpenAI and Google, open s...



## Assignment: Extend the System

Try one or more of these extensions:

1. **Adjustable similarity threshold** — Experiment with `threshold=0.1` (stricter) vs `threshold=0.35` (looser). How does it affect hit rate and answer quality?

2. **Cache TTL (Time-To-Live)** — Add an expiry timestamp to each cache entry. Stale entries (e.g., older than 7 days) should be evicted and re-fetched.

3. **Sub-query division** — Before checking the cache, use a GPT call to split compound questions (e.g., *"What was Uber and Lyft revenue in 2021?"*) into sub-queries. Check and populate the cache per sub-query.

4. **Cache analytics** — Track and display cache hit rate, average latency for hits vs misses, and the most-queried topics over a session.